In [1]:
import sys
import os

import qgis
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
from qgis.analysis import QgsNativeAlgorithms

sys.path.append('/usr/share/qgis/python/plugins/')
sys.path.append('/usr/share/qgis/python/')

QgsApplication.setPrefixPath('/usr', True)
app = QgsApplication([], False)
app.initQgis()

import processing
from processing.core.Processing import Processing
Processing.initialize()

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
# for alg in QgsApplication.processingRegistry().algorithms():
#         print(alg.id(), "--->", alg.displayName())

# Stop QGIS appllication
# app.exitQgis()
# app.exit()

True

In [2]:
from qgis.core import QgsProject, QgsApplication
# Get the project instance
project = QgsProject.instance()
# Print the current project file name (might be empty in case no projects have been loaded)
# print(project.fileName())

# Load  project
# project.read(f"/home/luca/Code/mappy.git/input_data/qgis_project.qgz")
project.read("/home/luca/Code/mappy.git/input_data/mars/mars.qgz")
print(project.fileName())

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/home/luca/Code/mappy.git/input_data/mars/mars.qgz


In [3]:
project.mapLayers()
cont = project.mapLayersByName("contacts")[0]
pols = project.mapLayersByName("map")[0]

In [16]:
cont.selectAll()
cont = processing.run("native:saveselectedfeatures", {'INPUT': cont, 'OUTPUT': 'TEMPORARY_OUTPUT'})['OUTPUT']

pols.selectAll()
pols = processing.run("native:saveselectedfeatures", {'INPUT': pols, 'OUTPUT': 'TEMPORARY_OUTPUT'})['OUTPUT']

In [19]:
dissolved = processing.run("native:dissolve", {'INPUT':pols,'FIELD':[],'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

dissolved #QgsProcessingUtils.generateTempFilename("dissolved.gpkg")}

<QgsVectorLayer: 'Dissolved' (memory)>

In [21]:
ex = processing.run("native:polygonfromlayerextent", {"INPUT":cont, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]

ex1 = processing.run("native:buffer",  {"INPUT":  ex, "DISTANCE":1e-2, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]


mask = processing.run("native:difference", {'INPUT':ex1,'OVERLAY':dissolved,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

In [25]:
mask.featureCount()

1

In [24]:
ff = list(mask.getFeatures())
[f.geometry() for f in ff]

[<QgsGeometry: MultiPolygon (((214.93895054746170103 13.79286015164277934, 214.93895054746170103 22.95824557708507285, 214.93943998229875092 22.96133574702882285, 214.94086037751793583 22.96412342960799791, 214.94307269493876333 22.96633574702882186, 214.94586037751793128 22.96775614224802453, 214.94895054746169194 22.96824557708507442, 240.36971396725047612 22.96824557708507442, 240.37280413719423677 22.96775614224802453, 240.37559181977340472 22.96633574702882186, 240.37780413719423223 22.96412342960799791, 240.37922453241341714 22.96133574702882285, 240.37971396725046702 22.95824557708507285, 240.37971396725046702 13.79286015164277934, 240.37922453241341714 13.78976998169902934, 240.37780413719423223 13.78698229911985429, 240.37559181977340472 13.78476998169903034, 240.37280413719423677 13.78334958647982766, 240.36971396725047612 13.78286015164277956, 214.94895054746169194 13.78286015164277956, 214.94586037751793128 13.78334958647982766, 214.94307269493876333 13.78476998169903034, 2

In [28]:
f = mask.getFeature(1)


print(pols.addFeature(f, QgsFeatureSink.FastInsert))



False


In [31]:
pols.fields()

In [7]:
# [2021-02-21 11:20:37] processing.run("native:dissolve", {'INPUT':'','FIELD':[],'OUTPUT':'TEMPORARY_OUTPUT'})
# dis = processing.run("native:dissolve", {"INPUT": pols, "OUTPUT":"TEMPORARY_OUTPUT"})


In [8]:
index = QgsSpatialIndex() # Spatial index

for ft in pols.getFeatures():
    
    index.insertFeature(ft)


<ipython-input-8-22e5363287d2>:5: DeprecationWarning: QgsSpatialIndex.insertFeature() is deprecated
  index.insertFeature(ft)


In [9]:


# pars = {'INPUT':cont2.id(),'OVERLAY':dissolved,'INPUT_FIELDS':[],'OVERLAY_FIELDS':[],'OVERLAY_FIELDS_PREFIX':'','OUTPUT':'TEMPORARY_OUTPUT'}
# cont = processing.run("native:intersection", pars)["OUTPUT"]

<QgsVectorLayer: 'Dissolved' (memory)>

In [10]:
# QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')    

In [11]:
outtable = []

for f in cont.getFeatures():
    fid = f.id()
    
    g = f.geometry()
    if g.isEmpty() or not f.isValid() or g.length() < 1e-16:
        continue
        
    
    for pid, p in enumerate(g.parts()):
        start = p[0]
        end = p[-1]

        outtable.append([fid, pid, [start, end]])
            
        
        


In [12]:
def find_polygons_containing_point(pt, index, polygons):
    ids = index.intersects(pt.boundingBox()) # polygons intersecting the point
    pp= polygons.getFeatures(ids)
    real = []
    for p, i in zip(pp, ids):
        if p.geometry().contains(QgsPointXY(pt)):
            real.append(p.id())
            
    return real
            

    

In [13]:
exteriors = []
cuttable = []
for fid, pid, (start, end) in outtable:
    mypols = []
    pstart = find_polygons_containing_point(start, index, pols)
    pend = find_polygons_containing_point(end, index, pols)
    mypols += pend + pstart
    
    if len(mypols) >1:
        cuttable.append([fid, pid, mypols])
        
    elif len(mypols) < 2:
        exteriors.append([fid, pid])

In [14]:
def get_sel(layer):
    return processing.run("native:saveselectedfeatures", {'INPUT':layer.id(),'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

In [15]:
# QgsProcessingFeatureSourceDefinition?

In [16]:
def clip_with_selection(lines, polygons):
    pars = {"INPUT":QgsProcessingFeatureSourceDefinition(lines.id(), True),
            "OVERLAY": QgsProcessingFeatureSourceDefinition(polygons.id(), True),
            "OUTPUT":"memory:"}
    
    
    return processing.run("native:difference", pars)

def clip_with_selection2(lines, polygons):
    lin = processing.run("native:saveselectedfeatures", {'INPUT':lines,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    pol = processing.run("native:saveselectedfeatures", {'INPUT':polygons,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    
    
    pars = {"INPUT":lin,
            "OVERLAY": pol,
            "OUTPUT":'TEMPORARY_OUTPUT'}
    
    
    return processing.run("native:difference", pars)["OUTPUT"]

def clip_external(lines, dissolved):
    lin = processing.run("native:saveselectedfeatures", {'INPUT':lines,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    return processing.run("native:clip", {"INPUT": lin, "OVERLAY":dissolved, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]
    
    

In [17]:
for i, (lineid, subid, polygons) in enumerate(cuttable):
    pols.removeSelection()
    cont.removeSelection()
    [pols.select(p) for p in polygons]
    cont.select(lineid)
    
    o = clip_with_selection2(cont, pols)
    
    if o.featureCount() > 0:
        f = o.getFeature(1)
        gg = f.geometry()

        cont.startEditing()
        cont.changeGeometry(lineid,gg)
        cont.commitChanges()

In [18]:
for i, (lineid, subid) in enumerate(exteriors):
    dissolved.removeSelection()
    cont.removeSelection()
    cont.select(lineid)
    
    o = clip_external(cont, dissolved)
    if o.featureCount() > 0:
        f = o.getFeature(1)
        gg = f.geometry()

        cont.startEditing()
        cont.changeGeometry(lineid,gg)
        cont.commitChanges()
    

In [19]:
cont

<QgsVectorLayer: 'Selected features' (memory)>

In [20]:
QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')    

<ipython-input-20-cacb76400b30>:1: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')


(0, '')

In [49]:
# QgsVectorFileWriter.writeAsVectorFormat(cont,"output",'utf-8',driverName='ESRI Shapefile')

<ipython-input-49-edf8a928671d>:1: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(cont,"output",'utf-8',driverName='ESRI Shapefile')


(0, '')

In [50]:
l = cont.getFeature(36)
p = pols.getFeature(30)
a

NameError: name 'a' is not defined

In [ ]:
pols.removeSelection()

In [ ]:
cont.select(36)
pols.select(30)


In [ ]:
pols.selectedFeatureIds()

In [ ]:
print([a.id() for a in QgsApplication.processingRegistry().algorithms() if "clip" in a.id()])

In [ ]:
[2021-02-15 11:39:59] processing.run("native:clip", {'INPUT':QgsProcessingFeatureSourceDefinition('contacts_7768adb1_5e8e_4a87_a7b2_aebd7bb4303e', True),
                                                     'OVERLAY':QgsProcessingFeatureSourceDefinition('map_62aaeba8_b51e_4242_82bc_ea14f8a87fbe', True),'OUTPUT':'TEMPORARY_OUTPUT'})